<div class='bar_title'></div>

*Decision Support Systems*

# Headline

student1, student2, student3, student,4 <br>

Winter Semester 19/20

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li></ul></div>

In [1]:
!pip install pulp
from pulp import *
import pandas as pd
import matplotlib.pyplot as plt

## Data Preparation

In [2]:
df_demand = pd.read_csv("https://github.com/wi3jmu/DSS1920/raw/master/Data/Bandwith_demand.csv", index_col=0)
df_supply = pd.read_csv("https://github.com/wi3jmu/DSS1920/raw/master/Data/Bandwith_supply.csv", index_col=0)

In [3]:
df_demand.head(3)

,region,area,usage
44851,B,BD,344.0
257130,C,CW,200.0
188337,C,CH,176.0


In [4]:
df_supply.head(3)

,area,supply
0,AL,303.335433
1,BB,300.613403
2,BD,342.146102


In [5]:
# lists
customers = list(df_demand.index)
regions = list(df_demand.region.unique())
areas = list(df_supply.area)

#dictionaries
supply = df_supply.set_index('area')['supply'].to_dict()
demand = df_demand['usage'].to_dict()

region_of_customer = [(r,c) for c,r in df_demand['region'].to_dict().items()]
customers_per_region = {k: len([1 for (r,c) in region_of_customer if k==r]) for k in regions}
area_of_customer = [(a,c) for c,a in df_demand['area'].to_dict().items()]
areas_of_region = [(a,r) for a,r in df_demand[['region','area']].drop_duplicates().set_index('area')['region'].to_dict().items()]
areas_of_customer = [(a,c) for (r,c) in region_of_customer for a in areas if (a,r) in areas_of_region]

## Implementation

In [6]:
# Create model
m1 = LpProblem("Bandwith", sense=pulp.LpMinimize)

# Variables
x = LpVariable.dicts("gb", areas_of_customer, lowBound=0, cat='Continuous')
z = LpVariable("z", lowBound=0, cat='Continuous')

# Objective
m1 += z

# Constraints
for r1 in regions:
    for r2 in regions:   
        m1 += lpSum([x[(a,c)] for (a,c) in areas_of_customer if ((a,r1) in areas_of_region)]+
                   [-x[(a,c)] for (a,c) in areas_of_customer if ((a,r2) in areas_of_region)]) <= z

for k in areas:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if a==k]) <= supply[k]
    
for k in customers:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if c==k]) >= 0.7*demand[k]
    
# Solve problem
m1.solve()
print (LpStatus[m1.status])
obj1 = value(m1.objective)

out = [sum([x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r) in areas_of_region)]) /
       customers_per_region[r] for r in regions]

print(obj1)
print(min(out))
print(sum(out)/len(out))

Optimal
1456.4124
46.487568
178.02469787466333


In [7]:
# Create model
m1 = LpProblem("Bandwith", sense=pulp.LpMaximize)

# Variables
x = LpVariable.dicts("gb", areas_of_customer, lowBound=0, cat='Continuous')
z = LpVariable("z", lowBound=0, cat='Continuous')

# Objective
m1 += z

# Constraints
for r in regions:
    m1 += lpSum([x[(a,c)] for (a,c) in areas_of_customer if ((a,r) in areas_of_region)]) >= z * customers_per_region[r]

for k in areas:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if a==k]) <= supply[k]
    
for k in customers:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if c==k]) >= 0.7*demand[k]
    
# Solve problem
m1.solve()
print (LpStatus[m1.status])
obj1 = value(m1.objective)

out = [sum([x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r) in areas_of_region)]) /
       customers_per_region[r] for r in regions]
diff = [sum([x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r1) in areas_of_region)]+
              [-x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r2) in areas_of_region)]) 
        for r1 in regions for r2 in regions]

print(max(diff))
print(obj1)
print(sum(out)/len(out))

Optimal
2062.992242
46.487568
223.69325955563974


In [8]:
print("Constraint  | Shadow Price | Slack")
for c in m1.constraints:
    print("%s | %f | %f"  %(c, m1.constraints[c].pi,m1.constraints[c].slack))

Constraint  | Shadow Price | Slack
_C1 | -0.000000 | -720.095320
_C2 | -0.000000 | -1598.116600
_C3 | -0.000000 | -495.109140
_C4 | -0.000000 | -256.575590
_C5 | -0.000000 | -1468.926600
_C6 | -0.000000 | -297.366370
_C7 | -0.000000 | -1016.982700
_C8 | -0.000000 | -85.881726
_C9 | -0.000000 | -251.712430
_C10 | -0.000000 | -1681.012300
_C11 | -0.000000 | -175.412430
_C12 | -0.000000 | -178.749730
_C13 | -0.000000 | -702.716370
_C14 | -0.000000 | -190.528900
_C15 | -0.000000 | -1217.162000
_C16 | -0.000000 | -136.337300
_C17 | -0.000000 | -602.255070
_C18 | -1.000000 | -0.000000
_C19 | -0.000000 | 81.435433
_C20 | -0.000000 | 0.000003
_C21 | -0.000000 | 0.000002
_C22 | -0.000000 | -0.000003
_C23 | -0.000000 | -0.000003
_C24 | -0.000000 | -0.000001
_C25 | -0.000000 | 0.000001
_C26 | -0.000000 | 0.000005
_C27 | -0.000000 | -0.000005
_C28 | -0.000000 | -0.000005
_C29 | -0.000000 | -0.000001
_C30 | -0.000000 | 0.000002
_C31 | -0.000000 | 0.000002
_C32 | -0.000000 | -0.000004
_C33 | -0.0000

In [9]:
# Create model
m1 = LpProblem("Bandwith", sense=pulp.LpMaximize)

# Variables
x = LpVariable.dicts("gb", areas_of_customer, lowBound=0, cat='Continuous')
z = LpVariable("z", lowBound=0, cat='Continuous')

# Objective
m1 += z

# Constraints
for r in regions:
    #if r!='Y':
        m1 += lpSum([x[(a,c)] for (a,c) in areas_of_customer if ((a,r) in areas_of_region)]) >= z * customers_per_region[r]

for k in areas:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if a==k]) <= supply[k]
    
for k in customers:
    m1 += lpSum([x[a,c] for (a,c) in areas_of_customer if c==k]) >= 0.7*demand[k]

# Solve problem
m1.solve()
print (LpStatus[m1.status])
obj1 = value(m1.objective)

out = [sum([x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r) in areas_of_region)]) /
       customers_per_region[r] for r in regions]
diff = [sum([x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r1) in areas_of_region)]+
              [-x[(a,c)].varValue for (a,c) in areas_of_customer if ((a,r2) in areas_of_region)]) 
        for r1 in regions for r2 in regions]

print(max(diff))
print(obj1)
print(sum(out)/len(out))

Optimal
2062.992242
46.487568
223.69325955563974
